# Making chloropleth maps in Altair

Here's a quick example of how to make a chloropleth map in Altair.  In this example, we'll work with a fairly large data set of baby names in France from 1900-2019, broken down by department.

To work with geographical data, we'll use the `geopandas`, which loads `pandas` dataframes, but with support for geographical outlines in the `geojson` format.  You can use these dataframes just as you would a regular `pandas` dataframe, but they will include that extra geographical outline data.

To get started, we'll need to import our libraries.

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
2109293,2,CÉLINE,1988,24,22
1080495,1,MANUEL,1917,34,17
1970290,2,AUDE,1986,57,24
843744,1,JEOFFREY,1986,59,7
1519266,1,SIMON,1992,58,8


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [3]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
66,66,Pyrénées-Orientales,"POLYGON ((2.16605 42.66392, 2.1762 42.65251, 2..."
52,52,Haute-Marne,"POLYGON ((4.67018 48.53189, 4.71801 48.54199, ..."
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.04907 46.75808, ..."
62,62,Pas-de-Calais,"POLYGON ((2.06771 51.00651, 2.0976 50.99843, 2..."
24,26,Drôme,"POLYGON ((4.80049 45.29836, 4.8588 45.30895, 4..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [4]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
3057537,08,Ardennes,"POLYGON ((4.23316 49.95775, 4.3081 49.96952, 4...",2,MARIE-NOËLLE,1961,08,10
2776341,25,Doubs,"POLYGON ((6.80701 47.5628, 6.81666 47.54792, 6...",2,LEÏLA,2016,25,5
771612,24,Dordogne,"POLYGON ((0.62974 45.71457, 0.65423 45.6887, 0...",1,JEAN-CHRISTOPHE,1968,24,17
3017718,72,Sarthe,"POLYGON ((-0.05453 48.382, -0.04463 48.37976, ...",2,MARIE-CÉCILE,1967,72,3
10829,81,Tarn,"POLYGON ((1.99017 44.14945, 2.0297 44.15704, 2...",1,ACHILLE,1908,81,3


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [5]:
grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum(numeric_only=True)
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [6]:
multi = alt.selection_multi()

name = 'LUCIEN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600).add_selection(
    multi
)

C:\Users\mlarn\AppData\Local\Temp\ipykernel_13548\2301346975.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  multi = alt.selection_multi()
C:\Users\mlarn\AppData\Local\Temp\ipykernel_13548\2301346975.py:8: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).properties(width=800, height=600).add_selection(


alt.Chart(...)

In [7]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [9]:
# Visualization 1

df_names = pd.read_csv('dpt2020.csv', sep=";", dtype={'dpt': str})
df_names.drop(df_names[df_names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df_names.drop(df_names[df_names.dpt == 'XX'].index, inplace=True)
gdf_depts = gpd.read_file('departements-version-simplifiee.geojson')

top_names = df_names.groupby('preusuel')['nombre'].sum().nlargest(10).index
bottom_names = df_names.groupby('preusuel')['nombre'].sum().nsmallest(10).index
all_names = top_names.union(bottom_names)
df_filtered = df_names[df_names['preusuel'].isin(all_names)].groupby(['preusuel', 'annais'])['nombre'].sum().reset_index()

top_names_year = df_names.groupby(['annais', 'preusuel'])['nombre'].sum().to_frame().reset_index()
top_names_year = top_names_year.groupby('annais', group_keys=False).apply(lambda x: x.sort_values('nombre', ascending=False)).reset_index()
top_names_year = top_names_year.groupby('annais').head(5)

selection = alt.selection_interval(encodings=['x'])

names_chart = alt.Chart(df_filtered).mark_line().encode(
    x=alt.X('annais:T', title='Year'),
    y=alt.Y('nombre:Q', title='Number'),
    color='preusuel:N',
    tooltip=[
        alt.Tooltip('preusuel:N', title='Name'), 
        alt.Tooltip('year(annais):O', title='Year'), 
        alt.Tooltip('nombre:Q', title='Number')]
).properties(
    width=500, height=200, title='Most popular names'
).add_selection(selection)

hist_names = alt.Chart(top_names_year).mark_bar().encode(
        alt.X("year(annais):O", title='Year'),
        alt.Y('sum(nombre):Q', title='Number'),
        color='preusuel:N',
        tooltip=[
            alt.Tooltip('preusuel:N', title='Name'),
            alt.Tooltip('year(annais):T', title='Year'),
            alt.Tooltip('nombre:Q', title='Number')],
    ).transform_filter(
        selection
    ).properties(width=500, height=200, title='Popular names per time period')


names_chart & hist_names

C:\Users\mlarn\AppData\Local\Temp\ipykernel_13548\3733497506.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_names_year = top_names_year.groupby('annais', group_keys=False).apply(lambda x: x.sort_values('nombre', ascending=False)).reset_index()
C:\Users\mlarn\AppData\Local\Temp\ipykernel_13548\3733497506.py:29: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(selection)


alt.VConcatChart(...)